In [21]:
import os
import pickle
import random
import socket
import struct

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from src.dataset.dataset_info import datasets
from src.graph.graph_construction.window_graph import create_weightless_window_graph
from src.graph.graph_measures import calculate_graph_measures
from src.graph.centralities.add_centralities import add_centralities, add_centralities_as_node_features


In [22]:
multi_class = True

use_port_in_address = True

generated_ips = False

# graph_type = "flow"
# graph_type = "window"
graph_type = "line"

window_size= 2000

sort_timestamp = False

# k_fold = None
# k_fold = 5

validation_size = 0.1
test_size = 0.1

use_node_features = True
cn_measures = ["betweenness", "degree", "pagerank", "closeness", "k_truss"]
# cn_measures = ["betweenness", "degree", "closeness"]

network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank', 'src_closeness', 'dst_closeness', 'src_k_truss', 'dst_k_truss']
# network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank']

In [23]:
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
name = "cic_ids_2017_5_percent"
# name = "cic_ids_2017"
# name = "cic_bot_iot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

dataset = datasets[name]

In [24]:
g_type = ""
if graph_type == "flow":
    g_type = "flow"
elif graph_type == "line":
    g_type = f"line_graph_{window_size}"
elif graph_type == "window":
    g_type = f"window_graph_{window_size}"
    
if multi_class:
    g_type += "__multi_class"
    
if use_node_features:
    g_type += "__n_feats"
    
# if k_fold:
#     g_type += f"__{k_fold}_fold"
    
if use_port_in_address:
    g_type += "__ports"
    
if generated_ips:
    g_type += "__generated_ips"
    
if sort_timestamp:
    g_type += "__sorted"
else:
    g_type += "__unsorted"
    
folder_path = os.path.join("datasets",name, g_type)
# folder_path = f"datasets/{name}/{g_type}"
folder_path

'datasets\\cic_ids_2017_5_percent\\line_graph_2000__multi_class__n_feats__ports__unsorted'

In [25]:
df = pd.read_parquet(dataset.path)

In [26]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,...,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66292,192.168.10.1-192.168.10.3-53-60671-17,192.168.10.3,60671.0,192.168.10.1,53.0,17.0,03/07/2017 10:23:37,30919.0,1.0,1.0,41.0,69.0,41.0,41.0,41.000,0.000000,69.0,69.0,69.000000,0.000000,3.557683e+03,64.685145,3.091900e+04,0.000000e+00,30919.0,30919.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,...,50.333333,16.165808,261.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,75.5,41.000,69.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41.0,1.0,69.0,-1.0,-1.0,0.0,32.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
420414,192.168.10.8-52.84.64.212-51938-443-6,52.84.64.212,443.0,192.168.10.8,51938.0,6.0,03/07/2017 11:20:18,3.0,1.0,1.0,6.0,6.0,6.0,6.0,6.000,0.000000,6.0,6.0,6.000000,0.000000,4.000000e+06,666666.666667,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,119.0,16360.0,0.0,20.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
393869,172.217.10.2-192.168.10.8-80-55154-6,192.168.10.8,55154.0,172.217.10.2,80.0,6.0,5/7/2017 1:53,115525809.0,16.0,14.0,442.0,122.0,358.0,0.0,27.625,88.112712,116.0,0.0,8.714286,30.920316,4.882026e+00,0.259682,3.983649e+06,4.849805e+06,9994305.0,83.0,116000000.0,7701720.6,10000000.0,36714.0,111000000.0,8508555.231,3716723.776,10000000.0,452.0,0.0,0.0,0.0,0.0,332.0,...,18.193548,66.307073,4396.627957,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,18.8,27.625,8.714286,0.0,0.0,0.0,0.0,0.0,0.0,16.0,442.0,14.0,122.0,8192.0,343.0,15.0,20.0,78974.81818,140027.3781,501173.0,36578.0,9976579.636,58066.24315,9994305.0,9801504.0,0,BENIGN,0
319307,192.168.10.1-192.168.10.3-53-61248-17,192.168.10.3,61248.0,192.168.10.1,53.0,17.0,03/07/2017 04:15:49,60594.0,1.0,1.0,45.0,235.0,45.0,45.0,45.000,0.000000,235.0,235.0,235.000000,0.000000,4.620920e+03,33.006568,6.059400e+04,0.000000e+00,60594.0,60594.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,108.333333,109.696551,12033.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,162.5,45.000,235.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45.0,1.0,235.0,-1.0,-1.0,0.0,20.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
419002,192.168.10.3-192.168.10.12-53-32968-17,192.168.10.12,32968.0,192.168.10.3,53.0,17.0,03/07/2017 09:40:12,310.0,2.0,2.0,68.0,412.0,34.0,34.0,34.000,0.000000,206.0,206.0,206.000000,0.000000,1.548387e+06,12903.225806,1.033333e+02,1.772465e+02,308.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000,0.000,1.0,1.0,0.0,0.0,0.0,0.0,40.0,...,102.800000,94.208280,8875.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,128.5,34.000,206.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,68.0,2.0,412.0,-1.0,-1.0,1.0,20.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0


In [27]:
cols_to_norm = list(set(list(df.columns))  - set(list([dataset.label_col, dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))

In [28]:
df[dataset.label_col].value_counts()

Label
0    113501
1     27754
Name: count, dtype: int64

In [29]:
if generated_ips:
    df[dataset.src_ip_col] = df[dataset.src_ip_col].apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [30]:
if sort_timestamp:
    df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)

In [31]:
if use_port_in_address:
    df[dataset.src_port_col] = df[dataset.src_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.src_ip_col] = df[dataset.src_ip_col] + ':' + df[dataset.src_port_col]

    df[dataset.dst_port_col] = df[dataset.dst_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.dst_ip_col] = df[dataset.dst_ip_col] + ':' + df[dataset.dst_port_col]

In [32]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,...,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Attack,Class
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66292,192.168.10.1-192.168.10.3-53-60671-17,192.168.10.3:60671,60671,192.168.10.1:53,53,17.0,03/07/2017 10:23:37,30919.0,1.0,1.0,41.0,69.0,41.0,41.0,41.000,0.000000,69.0,69.0,69.000000,0.000000,3.557683e+03,64.685145,3.091900e+04,0.000000e+00,30919.0,30919.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,32.0,...,50.333333,16.165808,261.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,75.5,41.000,69.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,41.0,1.0,69.0,-1.0,-1.0,0.0,32.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
420414,192.168.10.8-52.84.64.212-51938-443-6,52.84.64.212:443,443,192.168.10.8:51938,51938,6.0,03/07/2017 11:20:18,3.0,1.0,1.0,6.0,6.0,6.0,6.0,6.000,0.000000,6.0,6.0,6.000000,0.000000,4.000000e+06,666666.666667,3.000000e+00,0.000000e+00,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.000,6.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,1.0,6.0,119.0,16360.0,0.0,20.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
393869,172.217.10.2-192.168.10.8-80-55154-6,192.168.10.8:55154,55154,172.217.10.2:80,80,6.0,5/7/2017 1:53,115525809.0,16.0,14.0,442.0,122.0,358.0,0.0,27.625,88.112712,116.0,0.0,8.714286,30.920316,4.882026e+00,0.259682,3.983649e+06,4.849805e+06,9994305.0,83.0,116000000.0,7701720.6,10000000.0,36714.0,111000000.0,8508555.231,3716723.776,10000000.0,452.0,0.0,0.0,0.0,0.0,332.0,...,18.193548,66.307073,4396.627957,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,18.8,27.625,8.714286,0.0,0.0,0.0,0.0,0.0,0.0,16.0,442.0,14.0,122.0,8192.0,343.0,15.0,20.0,78974.81818,140027.3781,501173.0,36578.0,9976579.636,58066.24315,9994305.0,9801504.0,0,BENIGN,0
319307,192.168.10.1-192.168.10.3-53-61248-17,192.168.10.3:61248,61248,192.168.10.1:53,53,17.0,03/07/2017 04:15:49,60594.0,1.0,1.0,45.0,235.0,45.0,45.0,45.000,0.000000,235.0,235.0,235.000000,0.000000,4.620920e+03,33.006568,6.059400e+04,0.000000e+00,60594.0,60594.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,108.333333,109.696551,12033.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,162.5,45.000,235.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45.0,1.0,235.0,-1.0,-1.0,0.0,20.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0
419002,192.168.10.3-192.168.10.12-53-32968-17,192.168.10.12:32968,32968,192.168.10.3:53,53,17.0,03/07/2017 09:40:12,310.0,2.0,2.0,68.0,412.0,34.0,34.0,34.000,0.000000,206.0,206.0,206.000000,0.000000,1.548387e+06,12903.225806,1.033333e+02,1.772465e+02,308.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000,0.000,1.0,1.0,0.0,0.0,0.0,0.0,40.0,...,102.800000,94.208280,8875.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,128.5,34.000,206.000000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,68.0,2.0,412.0,-1.0,-1.0,1.0,20.0,0.00000,0.0000,0.0,0.0,0.000,0.00000,0.0,0.0,0,BENIGN,0


In [33]:
if multi_class:
    y = df[dataset.class_num_col]
else:
    y = df[dataset.label_col]

if sort_timestamp:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size)
else:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size, random_state=13, stratify=y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size, random_state=13, stratify=y_tr)

del df

In [34]:
if graph_type == "line" and use_node_features:
    add_centralities(df = X_train, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_val, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_test, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    cols_to_norm = list(set(cols_to_norm) | set(network_features))
    

calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> features_dicts: ('betweenness', 94072)
==>> features_dicts: ('degree', 94072)
==>> features_dicts: ('pagerank', 94072)
==>> features_dicts: ('closeness', 94072)
==>> features_dicts: ('k_truss', 94072)
calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> features_dicts: ('betweenness', 15120)
==>> features_dicts: ('degree', 15120)
==>> features_dicts: ('pagerank', 15120)
==>> features_dicts: ('closeness', 15120)
==>> features_dicts: ('k_truss', 15120)
calculated betweenness
calculated degree
calculated closeness
calculated pagerank
calculated k_truss
==>> features_dicts: ('betweenness', 16588)
==>> features_dicts: ('degree', 16588)
==>> features_dicts: ('pagerank', 16588)
==>> features_dicts: ('closeness', 16588)
==>> features_dicts: ('k_truss', 16588)


TypeError: unsupported operand type(s) for |=: 'list' and 'set'

In [38]:
scaler = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

cols_to_drop = list(set(list(X_train.columns)) - set(list([dataset.label_col, dataset.src_ip_col, dataset.dst_ip_col, dataset.class_num_col, 'h'])))
X_train.drop(cols_to_drop, axis=1, inplace=True)

X_val[cols_to_norm] = scaler.transform(X_val[cols_to_norm])
X_val['h'] = X_val[ cols_to_norm ].values.tolist()
X_val.drop(cols_to_drop, axis=1, inplace=True)

X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test[ cols_to_norm ].values.tolist()
X_test.drop(cols_to_drop, axis=1, inplace=True)

In [39]:
if graph_type == "window" or graph_type == "line":

    create_weightless_window_graph(
        df=X_train,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "training"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_val,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "validation"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_test,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "testing"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")

==>> number_of_groups: 58
==>> graph_measures of graph_0: {'number_of_nodes': 2000, 'number_of_edges': 7947, 'max_degree': 265, 'avg_degree': 7.947, 'density': 0.001987743871935968}
==>> graph_measures of graph_1: {'number_of_nodes': 2000, 'number_of_edges': 6775, 'max_degree': 271, 'avg_degree': 6.775, 'density': 0.0016945972986493246}
==>> graph_measures of graph_2: {'number_of_nodes': 2000, 'number_of_edges': 8242, 'max_degree': 275, 'avg_degree': 8.242, 'density': 0.0020615307653826915}
==>> graph_measures of graph_3: {'number_of_nodes': 2000, 'number_of_edges': 9074, 'max_degree': 269, 'avg_degree': 9.074, 'density': 0.0022696348174087045}
==>> graph_measures of graph_4: {'number_of_nodes': 2000, 'number_of_edges': 8668, 'max_degree': 258, 'avg_degree': 8.668, 'density': 0.0021680840420210106}
==>> graph_measures of graph_5: {'number_of_nodes': 2000, 'number_of_edges': 7148, 'max_degree': 289, 'avg_degree': 7.148, 'density': 0.0017878939469734866}
==>> graph_measures of graph_6: {

In [40]:
if graph_type == "flow":
	os.makedirs(folder_path, exist_ok=True)
	print(f"==>> X_train.shape: {X_train.shape}")
	print(f"==>> X_val.shape: {X_val.shape}")
	print(f"==>> X_test.shape: {X_test.shape}")


In [41]:
if graph_type == "flow":
    graph_name = "training_graph"

    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

In [42]:
if graph_type == "flow":
    graph_name = "validation_graph"

    G = nx.from_pandas_edgelist(X_val, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

In [43]:
if graph_type == "flow":
    graph_name = "testing_graph"
    
    G = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h', dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    # graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)